In [1]:
import network as layer
import torch.nn as nn

nc = 3
nz = 512

In [2]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.utils.data
lr     = 0.0002
beta1  = 0.5   
beta2  = 0.99     
imageSize = 128
batchSize = 32

outf = "./celeba_result/"
des_dir = "./celeba/"

dataset = dset.ImageFolder(root=des_dir,
                           transform=transforms.Compose([
                               transforms.CenterCrop(224),
                               transforms.Resize(imageSize),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size= batchSize,
                                         shuffle=True)

In [3]:
from custom_layers import *

class Generator(nn.Module):
    def make_dense(self, k_in, k_growth, n, options):
        layers = []
        for i in range(n):
            layers.append(Dense(layer.conv(k_in, k_growth, 3, 1, 1, **options)))
            k_in += k_growth
        return nn.Sequential(*layers)
    
    def __init__(self):
        super(Generator, self).__init__()
        options = {'leaky':False, 'bn':True, 'wn':False, 'pixel':False}
        layers = []
        k_growth = 12
        layers.append(layer.conv(nz, 256, 4, 1, 3, **options))
        
        layers.append(self.make_dense(256, k_growth, 3, options))
        layers.append(layer.conv(292, 256, 1, 1, 0, **options))
        layers.append(layer.conv(256, 256, 3, 1, 1, **options))
        # 4 x 4
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers.append(self.make_dense(256, k_growth, 3, options))
        layers.append(layer.conv(292, 256, 1, 1, 0, **options))
        layers.append(layer.conv(256, 256, 3, 1, 1, **options))
        # 8 x 8
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers.append(self.make_dense(256, k_growth, 3, options))
        layers.append(layer.conv(292, 256, 1, 1, 0, **options))
        layers.append(layer.conv(256, 256, 3, 1, 1, **options))
        # 16 x 16
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers.append(self.make_dense(256, k_growth, 3, options))
        layers.append(layer.conv(292, 128, 1, 1, 0, **options))
        layers.append(layer.conv(128, 128, 3, 1, 1, **options))
        # 32 x 32
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers.append(self.make_dense(128, k_growth, 3, options))
        layers.append(layer.conv(164, 64, 1, 1, 0, **options))
        layers.append(layer.conv(64, 64, 3, 1, 1, **options))
        # 64 x 64
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers.append(self.make_dense(64, k_growth, 3, options))
        layers.append(layer.conv(100, 32, 1, 1, 0, **options))
        layers.append(layer.conv(32, 32, 3, 1, 1, **options))
        # 128 x 128
        layers.append(layer.conv(32, nc, 1, 1, 0, leaky=True, bn=False, wn=True, pixel=True, only=True))
        
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.network(x)
    
netG = Generator()

In [4]:
class Discriminator(nn.Module):
    def make_dense(self, k_in, k_growth, n, options):
        layers = []
        for i in range(n):
            layers.append(Dense(layer.conv(k_in, k_growth, 3, 1, 1, **options)))
            k_in += k_growth
        return nn.Sequential(*layers)
    
    def __init__(self):
        super(Discriminator, self).__init__()
        options = {'leaky':True, 'bn':False, 'wn':True, 'pixel':False, 'gdrop':True}
        
        k_growth = 6
        layers = []
        # 128 x 128
        layers.append(self.make_dense(nc, k_growth, 6, options))
        layers.append(nn.AvgPool2d(kernel_size=2)) 
        # 64 x 64
        layers.append(self.make_dense(nc + 6 * k_growth, k_growth, 12, options))
        layers.append(nn.AvgPool2d(kernel_size=2)) 
        # 32 x 32
        layers.append(self.make_dense(nc + 18 * k_growth, k_growth, 12, options))
        layers.append(nn.AvgPool2d(kernel_size=2)) 
        # 16 x 16
        layers.append(self.make_dense(nc + 30 * k_growth, k_growth, 12, options))
        layers.append(nn.AvgPool2d(kernel_size=2)) 
        # 8 x 8
        layers.append(self.make_dense(nc + 42 * k_growth, k_growth, 18, options))
        layers.append(nn.AvgPool2d(kernel_size=2)) 
        # 4 x 4
        layers.append(self.make_dense(nc + 60 * k_growth, k_growth, 18, options))
        
        
        layers.append(minibatch_std_concat_layer())
        layers.append(layer.conv(nc + 78 * k_growth + 1, nc + 78 * k_growth, 3, 1, 1, **options))
        layers.append(layer.conv(nc + 78 * k_growth, nc + 78 * k_growth, 4, 1, 0, **options))
        layers.append(layer.linear(nc + 78 * k_growth, 1, sig=False, wn=True))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)
    
netD = Discriminator()

In [5]:
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torchvision.utils as vutils

In [6]:
criterion = nn.MSELoss()

input = torch.FloatTensor(batchSize, 3, imageSize,imageSize)
noise = torch.FloatTensor(batchSize, nz, 1, 1)
fixed_noise = torch.FloatTensor(batchSize, nz, 1, 1).normal_(0, 1)

label_real = torch.FloatTensor(batchSize)
label_fake = torch.FloatTensor(batchSize)

netD.cuda()
netG.cuda()
criterion.cuda()
input, label_real, label_fake = input.cuda(), label_real.cuda(), label_fake.cuda()
noise, fixed_noise = noise.cuda(), fixed_noise.cuda()
label_real.resize_(batchSize, 1).fill_(1)
label_fake.resize_(batchSize, 1).fill_(0)
label_real = Variable(label_real)
label_fake = Variable(label_fake)
print()

In [7]:
fixed_noise = Variable(fixed_noise)

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, beta2))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, beta2))
schedulerD = optim.lr_scheduler.MultiStepLR(optimizerD, milestones=[4, 7, 11, 17], gamma=0.87)
schedulerG = optim.lr_scheduler.MultiStepLR(optimizerG, milestones=[4, 7, 11, 17], gamma=0.87)
result_dict = {}
loss_D,loss_G,score_D,score_G1,score_G2 = [],[],[],[],[]

In [ ]:
import numpy as np
_d_ = None
def add_noise(x, d_fake):
    global _d_
    if _d_ is not None:
        _d_ = _d_ * 0.9 + torch.mean(d_fake).data[0] * 0.1
        strength = 0.2 * max(0, _d_ - 0.5)**2
        z = np.random.randn(*x.size()).astype(np.float32) * strength
        z = Variable(torch.from_numpy(z)).cuda()
        return x + z
    else:
        _d_ = 0.0
        return x


In [ ]:
niter = 100
d_fake_save = None
for epoch in range(niter):
    schedulerD.step()
    schedulerG.step()
    for i, data in enumerate(dataloader, 0):

        # train D
        netD.zero_grad()
        real_cpu, _ = data
        batch_size = real_cpu.size(0)

        real_cpu = real_cpu.cuda()
        input.resize_as_(real_cpu).copy_(real_cpu)

        inputv = Variable(input)
        inputv = add_noise(inputv, d_fake_save)
        
        d_real = netD(inputv)
        d_real_mean = d_real.data.mean()
        
        noise.resize_(batch_size, nz, 1, 1).normal_(0, 1)
        noisev = Variable(noise)
        fake = netG(noisev)
        
        d_fake_save = d_fake = netD(fake.detach())
        d_fake_mean = d_fake.data.mean()
        
        loss_d = criterion(d_real, label_real) + criterion(d_fake, label_fake)
        loss_d.backward()
        optimizerD.step()
        
        # train G
        netG.zero_grad()
        d_fake = netD(fake)
        loss_g = criterion(d_fake, label_real.detach())
        loss_g.backward()
        optimizerG.step()

        if i%250 == 0:
            fake = netG(fixed_noise)
            vutils.save_image(fake.data,
                    '%s/fake_samples_epoch_%03dstep_%04d.png' % (outf, epoch, i),
                    normalize=True)
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f'
                  % (epoch, niter, i, len(dataloader),
                     loss_d.data[0], loss_g.data[0], d_real_mean, d_fake_mean))

[0/100][0/6330] Loss_D: 1.2027 Loss_G: 1.1229 D(x): -0.0933 D(G(z)): -0.0815
[0/100][250/6330] Loss_D: 0.4413 Loss_G: 0.3645 D(x): 0.5307 D(G(z)): 0.3967
[0/100][500/6330] Loss_D: 0.5284 Loss_G: 0.2709 D(x): 0.4723 D(G(z)): 0.4965
[0/100][750/6330] Loss_D: 0.5209 Loss_G: 0.2421 D(x): 0.4970 D(G(z)): 0.5173
[0/100][1000/6330] Loss_D: 0.5134 Loss_G: 0.2552 D(x): 0.4856 D(G(z)): 0.4987
[0/100][1250/6330] Loss_D: 0.5138 Loss_G: 0.2429 D(x): 0.4987 D(G(z)): 0.5123
[0/100][1500/6330] Loss_D: 0.5106 Loss_G: 0.2516 D(x): 0.4906 D(G(z)): 0.5010
[0/100][1750/6330] Loss_D: 0.5160 Loss_G: 0.2438 D(x): 0.4987 D(G(z)): 0.5144
[0/100][2000/6330] Loss_D: 0.5088 Loss_G: 0.2517 D(x): 0.4940 D(G(z)): 0.5027
[0/100][2250/6330] Loss_D: 0.5069 Loss_G: 0.2514 D(x): 0.4944 D(G(z)): 0.5012
[0/100][2500/6330] Loss_D: 0.5098 Loss_G: 0.2495 D(x): 0.4872 D(G(z)): 0.4967
[0/100][2750/6330] Loss_D: 0.5088 Loss_G: 0.2534 D(x): 0.4852 D(G(z)): 0.4935
[0/100][3000/6330] Loss_D: 0.5057 Loss_G: 0.2465 D(x): 0.5018 D(G(z)

[4/100][250/6330] Loss_D: 0.5004 Loss_G: 0.2504 D(x): 0.4996 D(G(z)): 0.4999
[4/100][500/6330] Loss_D: 0.5005 Loss_G: 0.2511 D(x): 0.5005 D(G(z)): 0.5010
[4/100][750/6330] Loss_D: 0.5012 Loss_G: 0.2493 D(x): 0.4982 D(G(z)): 0.4993
[4/100][1000/6330] Loss_D: 0.5005 Loss_G: 0.2523 D(x): 0.4984 D(G(z)): 0.4989
[4/100][1250/6330] Loss_D: 0.5009 Loss_G: 0.2493 D(x): 0.4978 D(G(z)): 0.4987
[4/100][1500/6330] Loss_D: 0.5004 Loss_G: 0.2503 D(x): 0.4988 D(G(z)): 0.4991
[4/100][1750/6330] Loss_D: 0.5004 Loss_G: 0.2486 D(x): 0.5009 D(G(z)): 0.5013
[4/100][2000/6330] Loss_D: 0.5004 Loss_G: 0.2491 D(x): 0.4980 D(G(z)): 0.4984
[4/100][2250/6330] Loss_D: 0.4999 Loss_G: 0.2497 D(x): 0.4987 D(G(z)): 0.4985
[4/100][2500/6330] Loss_D: 0.5010 Loss_G: 0.2508 D(x): 0.4985 D(G(z)): 0.4994
[4/100][2750/6330] Loss_D: 0.5009 Loss_G: 0.2479 D(x): 0.5001 D(G(z)): 0.5010
[4/100][3000/6330] Loss_D: 0.5009 Loss_G: 0.2487 D(x): 0.4997 D(G(z)): 0.5006
[4/100][3250/6330] Loss_D: 0.5012 Loss_G: 0.2482 D(x): 0.4997 D(G(z

[8/100][500/6330] Loss_D: 0.4944 Loss_G: 0.2673 D(x): 0.5094 D(G(z)): 0.5036
[8/100][750/6330] Loss_D: 0.4969 Loss_G: 0.2631 D(x): 0.5169 D(G(z)): 0.5132
[8/100][1000/6330] Loss_D: 0.4915 Loss_G: 0.2651 D(x): 0.5167 D(G(z)): 0.5078
[8/100][1250/6330] Loss_D: 0.4778 Loss_G: 0.2650 D(x): 0.5368 D(G(z)): 0.5129
[8/100][1500/6330] Loss_D: 0.4733 Loss_G: 0.2516 D(x): 0.5070 D(G(z)): 0.4796
[8/100][1750/6330] Loss_D: 0.4734 Loss_G: 0.2779 D(x): 0.5066 D(G(z)): 0.4794
[8/100][2000/6330] Loss_D: 0.4611 Loss_G: 0.3456 D(x): 0.5758 D(G(z)): 0.5298
[8/100][2250/6330] Loss_D: 0.4366 Loss_G: 0.3020 D(x): 0.5393 D(G(z)): 0.4732
[8/100][2500/6330] Loss_D: 0.4965 Loss_G: 0.2603 D(x): 0.5604 D(G(z)): 0.5504
[8/100][2750/6330] Loss_D: 0.4904 Loss_G: 0.2733 D(x): 0.5475 D(G(z)): 0.5343
[8/100][3000/6330] Loss_D: 0.4963 Loss_G: 0.3069 D(x): 0.5340 D(G(z)): 0.5281
[8/100][3250/6330] Loss_D: 0.4781 Loss_G: 0.2623 D(x): 0.5527 D(G(z)): 0.5271
[8/100][3500/6330] Loss_D: 0.4542 Loss_G: 0.3004 D(x): 0.4984 D(G(